In [53]:
%pip install chess
%pip installl tensorflow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
ERROR: unknown command "installl" - maybe you meant "install"
Note: you may need to restart the kernel to use updated packages.


In [1]:
import requests
import gzip
import shutil

url = "https://storage.googleapis.com/chesspic/datasets/2021-07-31-lichess-evaluations-37MM.db.gz"
local_filename = "2021-07-31-lichess-evaluations-37MM.db.gz"

with requests.get(url, stream=True) as r:
    with open(local_filename, 'wb') as f:
        shutil.copyfileobj(r.raw, f)

with gzip.open(local_filename, 'rb') as f_in:
    with open("2021-07-31-lichess-evaluations-37MM.db", 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

print("Download and decompression complete.")

Download and decompression complete.


In [55]:
%pip install peewee
%pip install scikit-learn
import base64
import chess
import numpy as np
from peewee import Model, IntegerField, TextField, BlobField, FloatField, SqliteDatabase
from sklearn.model_selection import train_test_split

db = SqliteDatabase('2021-07-31-lichess-evaluations-37MM.db')
class Evaluations(Model):
    id = IntegerField()
    fen = TextField()
    binary = BlobField()
    eval = FloatField()

    class Meta:
        database = db

    def binary_base64(self):
        return base64.b64encode(self.binary)

# Connect to the database
db.connect()

# Function to fetch all data from the database

def split_dims(board):
    board3d = np.zeros((14, 8, 8), dtype=np.int8)

    for piece in chess.PIECE_TYPES:
        for square in board.pieces(piece, chess.WHITE):
            idx = np.unravel_index(square, (8, 8))
            board3d[piece - 1][7 - idx[0]][idx[1]] = 1
        for square in board.pieces(piece, chess.BLACK):
            idx = np.unravel_index(square, (8, 8))
            board3d[piece + 5][7 - idx[0]][idx[1]] = 1

    return board3d

# def fetch_all_data():
#     boards = []
#     evaluations = []

#     for evaluation in Evaluations.select():
#         board = chess.Board(evaluation.fen)
#         board_3d = split_dims(board)
#         # Normalize the evaluation score
#         eval = max(evaluation.eval, -15)
#         eval = min(eval, 15)

#         boards.append(board_3d)
#         evaluations.append(eval)

#     return np.array(boards), np.array(evaluations)

# Fetch data from the database
# x_train, y_train = fetch_all_data()

# Function to fetch the first ten data points from the database
def fetch_first_ten_data():
    boards = []
    evaluations = []

    query = Evaluations.select().limit(100000)
    for evaluation in query:
        # Parse FEN string to chess board
        board = chess.Board(evaluation.fen)
        board_3d = split_dims(board)

        # Normalize the evaluation score
        eval = max(evaluation.eval, -15)
        eval = min(eval, 15)

        boards.append(board_3d)
        evaluations.append(eval)

    return np.array(boards), np.array(evaluations)

x_data, y_data = fetch_first_ten_data()

# Split data into training and test sets
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)

np.savez('random_dataset.npz', b=x_train, v=y_train, allow_pickle=True)

print("Dataset saved successfully.")

# Close the database connection
db.close()


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Dataset saved successfully.


True

In [56]:
import chess
import chess.engine
import random
import numpy
import tensorflow  # type: ignore

In [57]:
def random_board(max_depth=200):
  board = chess.Board()
  depth = random.randrange(0, max_depth)

  for _ in range(depth):
    all_moves = list(board.legal_moves)
    random_move = random.choice(all_moves)
    board.push(random_move)
    if board.is_game_over():
      break

  return board


# this function will create our f(x) (score)
def stockfish(board, depth):
    with chess.engine.SimpleEngine.popen_uci('./stockfish') as sf:
        result = sf.analyse(board, chess.engine.Limit(depth=depth))
        score = result['score'].white().score()
        return score if score is not None else 0

In [58]:
board = random_board()
board
stockfish(board, 10)

-459

In [59]:
squares_index = {
  'a': 0,
  'b': 1,
  'c': 2,
  'd': 3,
  'e': 4,
  'f': 5,
  'g': 6,
  'h': 7
}


# example: h3 -> 17
def square_to_index(square):
  letter = chess.square_name(square)
  return 8 - int(letter[1]), squares_index[letter[0]]


def split_dims(board):
  # this is the 3d matrix
  board3d = numpy.zeros((14, 8, 8), dtype=numpy.int8)

  # here we add the pieces's view on the matrix
  for piece in chess.PIECE_TYPES:
    for square in board.pieces(piece, chess.WHITE):
      idx = numpy.unravel_index(square, (8, 8))
      board3d[piece - 1][7 - idx[0]][idx[1]] = 1
    for square in board.pieces(piece, chess.BLACK):
      idx = numpy.unravel_index(square, (8, 8))
      board3d[piece + 5][7 - idx[0]][idx[1]] = 1

  # add attacks and valid moves too
  # so the network knows what is being attacked
  aux = board.turn
  board.turn = chess.WHITE
  for move in board.legal_moves:
      i, j = square_to_index(move.to_square)
      board3d[12][i][j] = 1
  board.turn = chess.BLACK
  for move in board.legal_moves:
      i, j = square_to_index(move.to_square)
      board3d[13][i][j] = 1
  board.turn = aux

  return board3d

In [60]:
# # Function to generate random chess boards and evaluations
# def generate_data(num_samples=1000):
#     boards = []
#     evaluations = []

#     for _ in range(num_samples):
#         board = random_board()
#         evaluation = stockfish(board, 10) / 1000.0  # Normalize evaluation score
#         if evaluation is not None:
#             boards.append(split_dims(board))
#             evaluations.append(evaluation)

#     return numpy.array(boards), numpy.array(evaluations)

# # Function to save the dataset into an .npz file
# def save_dataset(filename, boards, evaluations):
#     numpy.savez(filename, b=boards, v=evaluations, allow_pickle=True)

# # Generate data
# x_train, y_train = generate_data(num_samples=10)
# print( y_train)

# # Save the dataset into an .numpyz file
# save_dataset('random_dataset.npz', x_train, y_train)

# print("Dataset saved successfully.")

In [61]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split

def build_model(conv_size, conv_depth):
    board3d = layers.Input(shape=(14, 8, 8))
    x = board3d
    for _ in range(conv_depth):
        x = layers.Conv2D(filters=conv_size, kernel_size=3, padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)
    x = layers.Flatten()(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dense(1, activation='tanh')(x)
    return models.Model(inputs=board3d, outputs=x)

In [89]:
def build_model_residual(conv_size, conv_depth):
    board3d = layers.Input(shape=(14, 8, 8))

    # Initial Convolutional Layer
    x = layers.Conv2D(filters=conv_size, kernel_size=3, padding='same')(board3d)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    # Adding Residual Blocks
    for _ in range(conv_depth):
        previous = x
        x = layers.Conv2D(filters=conv_size, kernel_size=3, padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)
        x = layers.Conv2D(filters=conv_size, kernel_size=3, padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Add()([x, previous])
        x = layers.Activation('relu')(x)

    # Fully Connected Layers
    x = layers.Flatten()(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dense(1, activation='tanh')(x)

    return models.Model(inputs=board3d, outputs=x)

In [90]:
model = build_model(32, 4)

In [91]:
import tensorflow.keras.callbacks as callbacks

def get_dataset():
    container = numpy.load('random_dataset.npz', allow_pickle=True)
    b, v = container.get('b'), container.get('v')
    v = numpy.array(v) 
    v = numpy.where(v == None, 0, v)
    v = numpy.asarray(v / abs(v).max() / 2 + 0.5, dtype=numpy.float32) # normalization (0 - 1)
    return b, v


x_train, y_train = get_dataset()
x_train.transpose()
print(x_train.shape)
print(y_train.shape)

(80000, 14, 8, 8)
(80000,)


In [92]:
model = build_model(32, 4)
model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),  # Adjusted learning rate
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=[tf.keras.metrics.MeanAbsoluteError()])

checkpoint_filepath = '/tmp/checkpoint/model.keras'
model_checkpointing_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only=True,
)

history = model.fit(x_train, y_train,
                    batch_size=64,  # Adjusted batch size
                    epochs=1000,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=[ReduceLROnPlateau(monitor='val_loss', patience=10),
                               EarlyStopping(monitor='val_loss', patience=15, min_delta=1e-4),
                               model_checkpointing_callback])

loss, mae = model.evaluate(x_test, y_test)
print(f'Test Loss: {loss}')
print(f'Test MAE: {mae}')
model.save('model.h5')

Epoch 1/1000
625/625 ━━━━━━━━━━━━━━━━━━━━ 160s 251ms/step - loss: 0.1491 - mean_absolute_error: 0.4950 - val_loss: 4.3232 - val_mean_absolute_error: 4.7876 - learning_rate: 1.0000e-04
Epoch 2/1000
625/625 ━━━━━━━━━━━━━━━━━━━━ 177s 283ms/step - loss: 0.1460 - mean_absolute_error: 0.4913 - val_loss: 4.3232 - val_mean_absolute_error: 4.7876 - learning_rate: 1.0000e-04
Epoch 3/1000
224/625 ━━━━━━━━━━━━━━━━━━━━ 1:45 262ms/step - loss: 0.1477 - mean_absolute_error: 0.4940

KeyboardInterrupt: 

In [88]:
y_pred = model.predict(x_test)

# Calculate the accuracy with a threshold of 0.2
threshold = 0.2
correct_predictions = np.abs(y_pred.flatten() - y_test) <= threshold
accuracy = np.mean(correct_predictions)

print(f'Accuracy: {accuracy}')

from tensorflow.keras import models
model = models.load_model('model.h5')

625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step
Accuracy: 0.069


In [67]:
# used for the minimax algorithm
def minimax_eval(board):
  board3d = split_dims(board)
  board3d = numpy.expand_dims(board3d, 0)
  return model(board3d)[0][0]


def minimax(board, depth, alpha, beta, maximizing_player):
  if depth == 0 or board.is_game_over():
    return minimax_eval(board)

  if maximizing_player:
    max_eval = -numpy.inf
    for move in board.legal_moves:
      board.push(move)
      eval = minimax(board, depth - 1, alpha, beta, False)
      board.pop()
      max_eval = max(max_eval, eval)
      alpha = max(alpha, eval)
      if beta <= alpha:
        break
    return max_eval
  else:
    min_eval = numpy.inf
    for move in board.legal_moves:
      board.push(move)
      eval = minimax(board, depth - 1, alpha, beta, True)
      board.pop()
      min_eval = min(min_eval, eval)
      beta = min(beta, eval)
      if beta <= alpha:
        break
    return min_eval


# this is the actual function that gets the move from the neural network
def get_ai_move(board, depth):
  max_move = None
  max_eval = -numpy.inf

  for move in board.legal_moves:
    board.push(move)
    eval = minimax(board, depth - 1, -numpy.inf, numpy.inf, False)
    board.pop()
    if eval > max_eval:
      max_eval = eval
      max_move = move

  return max_move

In [68]:
# Testing code AI(white) vs Stockfish(black)
board = chess.Board()
from IPython.display import clear_output

with chess.engine.SimpleEngine.popen_uci('./stockfish') as engine:
    while True:
        clear_output(wait=True)
        move = get_ai_move(board, 1)
        board.push(move)
        print(move)
        print(f'\n{board}')
        if board.is_game_over():
            print('game_over')
            break
        move = engine.analyse(board, chess.engine.Limit(time=0.1), info=chess.engine.INFO_PV)['pv'][0]
        board.push(move)
        print(f'\n{board}')
        if board.is_game_over():
            print('game_over')
            break

f1f5

r . b . . r k .
p . p . . p p p
p . n p . . . .
. . . . p R . .
. . . . P . . q
. . N . . . . .
P P P P . . n .
R . B K . . b .

r . b . . r k .
p . p . . p p p
p . n p . . . .
. . . . p R . .
. . . . P . . .
. . N . . . . .
P P P P . . n .
R . B K q . b .
game_over


In [69]:
# Move by move testing code AI(white) vs Stockfish(black)
import os
import chess.engine

# Get the absolute path to the stockfish executable
stockfish_path = os.path.abspath('./stockfish')

board = chess.Board()

with chess.engine.SimpleEngine.popen_uci(stockfish_path) as engine:
    while True:
        move = get_ai_move(board, 1)
        board.push(move)
        print(f'\n{board}')
        if board.is_game_over():
            print('game_over')
            break
        move = engine.analyse(board, chess.engine.Limit(time=1), info=chess.engine.INFO_PV)['pv'][0]
        board.push(move)
        print(f'\n{board}')
        if board.is_game_over():
            print('game_over')
            break


r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R

r n b q k b n r
p p p p . p p p
. . . . . . . .
. . . . p . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R

r n b q k b n r
p p p p . p p p
. . . . . . . .
. . . . p . . Q
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B . K B N R

r . b q k b n r
p p p p . p p p
. . n . . . . .
. . . . p . . Q
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B . K B N R

r . b q k b n r
p p p p . p p p
. . n . . . . .
. . . . p . . Q
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B K . B N R

r . b q k b . r
p p p p . p p p
. . n . . n . .
. . . . p . . Q
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B K . B N R

r . b q k b . r
p p p p . p p p
. . n . . n . .
. . . . p . . Q
. . . . P . . .
. . . . . P . .
P P P P . . P P
R N B K . B N R

r . b q k b . r
p p p p . p p p
. . n . . . . .
. . . . p . . n
. . . . P . . .
. . . . . P . .


In [70]:
board = chess.Board()
from IPython.display import clear_output

with chess.engine.SimpleEngine.popen_uci('./stockfish') as engine:
    while True:
        clear_output(wait=True)
        move = get_ai_move(board, 1)
        board.push(move)
        print(move)
        print(f'\n{board}')
        if board.is_game_over():
            print('game_over')
            break
        input_var = input()
        move = chess.Move.from_uci(input_var)
        board.push(move)
        print(move)
        print(f'\n{board}')
        if board.is_game_over():
            print('game_over')
            break

e2e4

r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R


InvalidMoveError: expected uci string to be of length 4 or 5: ''